# Semantic Layer Consumption Example

**Purpose:** Demonstrate how the SQL semantic layer enables reproducible analytical consumption with minimal data wrangling.

## What This Notebook Does
- Load semantic views (from CSV or PostgreSQL)
- Build analysis-ready dataset at heat level
- Validate grain and data quality
- Show two simple consumption examples

## What This Notebook Does NOT Do
- SPC or control charts → SC02+
- Root-cause analysis → SC02+
- Statistical modeling → SC02+
- Deep domain analysis

**Key Insight:** The simplicity is intentional. Complexity is handled upstream in the data model.

## Section 0: Setup & Configuration

Choose your data source:
- **CSV** (default): No dependencies, works offline
- **PostgreSQL**: See SQL queries in action, requires DB running

Both return data in the same format.

In [1]:
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

sys.path.insert(0, str(Path.cwd().parent / 'src'))

from loaders import load_semantic_views, resolve_repo_root, load_database_url
from plots import plot_uts_histogram, plot_uts_boxplot

plt.style.use('ggplot')


ModuleNotFoundError: No module named 'sqlalchemy'

## Section 1: Load Semantic Views

Load three semantic views:
1. `v_heats_by_alloy`: 1 row per heat (alloy assignment)
2. `v_heats_by_final_product`: 1 row per heat (product metadata)
3. `v_lab_values_by_heats`: multiple rows per heat (lab results)
4. `v_analysis_dataset` : analysis ready view for comparsion with df built qith pandas

The same code works for both CSV and PostgreSQL.

In [ ]:
repo_root = resolve_repo_root()
database_url = load_database_url(repo_root)
DATA_SOURCE = 'postgresql'

loaded = load_semantic_views(
    data_source=DATA_SOURCE,
    repo_root=repo_root,
    database_url=database_url
)

df_heats_alloy = loaded['df_heats_alloy']
df_final_prod = loaded['df_final_prod']
df_lab = loaded['df_lab']
df_sql = loaded['df_sql']

print('✓ Loaded semantic views:\n')
print(f'  - source: {loaded['data_source']}')
print(f'  - v_heats_by_alloy: {len(df_heats_alloy)} rows')
print(f'  - v_final_product: {len(df_final_prod)} rows')
print(f'  - v_lab_values: {len(df_lab)} rows')
print(f'  - v_lab_values: {len(df_sql)} rows\n')

## Section 2: Grain Validation

**Critical:** Verify the semantic layer respects heat-level grain contracts.

Expected grain:
- `v_heats_by_alloy`: 1 row per heat
- `v_final_product`: 1 row per heat
- `v_lab_values`: 1 row per (heat, test_name, session_type)

This prevents silent errors in joins.

In [ ]:
# ============================================================================
# SECTION 2: Grain Validation
# ============================================================================

print('\n' + '=' * 80)
print('GRAIN VALIDATION (Core to SC01)')
print('=' * 80 + '\n')

# Validate v_heats_by_alloy
grain_alloy = df_heats_alloy.groupby('heat_id').size()
assert grain_alloy.max() == 1, 'ERROR: Duplicates in v_heats_by_alloy'
print(f'✓ v_heats_by_alloy: 1 row per heat ({len(df_heats_alloy)} heats)')

# Validate v_final_product
grain_product = df_final_prod.groupby('heat_id').size()
assert grain_product.max() == 1, 'ERROR: Duplicates in v_final_product'
print(f'✓ v_final_product: 1 row per heat ({len(df_final_prod)} heats)')

# Validate v_lab_values
lab_key = df_lab.groupby(['heat_id', 'test_name']).size()
assert lab_key.max() == 1, 'ERROR: Duplicates in v_lab_values'
print(f'✓ v_lab_values: No duplicates within (heat, test, session)')

# Check alignment
common_heats = set(df_heats_alloy['heat_id']) & set(df_final_prod['heat_id'])
print(f'\n✓ {len(common_heats)} heats in both alloy and product views')
print('\n✓ GRAIN CONTRACTS VALIDATED')

## Section 3: Build Analysis-Ready Dataset

### Two Approaches to the Same Result

In this section, we demonstrate the core value proposition of a semantic layer:

**Approach 1 (Pandas):** Explicitly construct the analysis dataset by pivoting lab values and joining semantic views. This shows the *logic* behind data construction—each step is visible and debuggable.

**Approach 2 (SQL View):** Load pre-built dataset directly from `v_analysis_dataset` view. This demonstrates the *efficiency* gain: the analyst writes one line of SQL instead of multiple pandas operations.

### Why This Matters

With a well-designed semantic layer:
- **Reduced complexity:** Analysts work with curated views, not raw tables
- **Formal data contracts:** Grain, column names, and transformations are standardized
- **Less boilerplate:** No need to rewrite pivot/join logic across projects
- **Single source of truth:** Data definition lives in database, not notebooks

The comparison in **Section 3B** validates that both approaches produce identical results, proving the semantic layer abstraction is correct and trustworthy.

**Approach 1 - Pandas:**

Create 1 row per heat by:
1. Pivoting lab results (test_name → columns)
2. Inner join on heat_id

This replaces multi-file spreadsheet merges.

In [ ]:
# ============================================================================
# SECTION 3: Build Analysis-Ready Dataset
# ============================================================================

print('\n' + '=' * 80)
print('BUILDING ANALYSIS-READY DATASET')
print('=' * 80 + '\n')

# Pivot lab results wide
lab_wide = (
    df_lab.pivot_table(
        index='heat_id',
        columns='test_name',
        values='test_value',
        aggfunc='mean'
    )
    .rename(columns={
        'UTS_MPa': 'uts_value',
        'YS_MPa': 'ys_value',
        'EL_percent': 'elongation_value',
        'Thickness measurement': 'thickness_mm'
    })
    .reset_index()
)

print(f'Pivoted lab data: {len(lab_wide)} heats with lab results')

# Join alloy + product + lab
df = (
    df_heats_alloy
    .merge(df_final_prod, on=['heat_id','heat_num','alloy_code'])
    .merge(lab_wide, on='heat_id')
)

# Select columns
df = df[[
    'heat_id', 'alloy_code', 'product_type', 'base_temper', 'h_level','spec_thickness',
    'uts_value', 'ys_value', 'elongation_value'
]]

print(f'\nFinal dataset: {len(df)} heats, {len(df.columns)} features')
print(f'Grain: 1 row per heat (validated)')
print(f'\nFirst 5 rows:')
display(df.head())


**SQL: `v_analysis_dataset`:**
- Single query with LEFT JOINs
- Subqueries for each test type (UTS, YS, elongation)
- Same result: 1 row per heat


In [ ]:
display(df_sql.head())

## Section 3B: Pandas vs SQL - Comparing Approaches

### Why Compare?

Both Pandas and SQL can build the same analysis dataset. Comparing them validates:
- Pandas transformations are correct
- SQL view logic is equivalent
- Results are reproducible in either approach

In [ ]:
# Comparison: Pandas vs SQL Approach
print('\n' + '=' * 80)
print('COMPARISON: Pandas vs SQL Approach')
print('=' * 80 + '\n')

if df_sql is not None:  # PostgreSQL mode
    # Sort both datasets by heat_id for comparison
    df_sql_sorted = df_sql.sort_values('heat_id').reset_index(drop=True)
    df_pandas_sorted = df.sort_values('heat_id').reset_index(drop=True)
    
    # Find common columns
    common_cols = list(set(df_sql_sorted.columns) & set(df_pandas_sorted.columns))
    
    # Compare
    if df_sql_sorted[common_cols].equals(df_pandas_sorted[common_cols]):
        print('✓ Pandas and SQL produce IDENTICAL results')
        print(f'  Pandas approach: {len(df)} heats, {len(df.columns)} features')
        print(f'  SQL approach: {len(df_sql)} heats, {len(df_sql.columns)} features')
    else:
        print('✗ WARNING: Results differ!')
else:
    print('(Skipped: CSV mode, no SQL comparison available)')

## Section 4: Example Analysis #1 - Simple Segmentation

Filter → GroupBy → Aggregate

This is a common consumption pattern.

In [ ]:
# ============================================================================
# SECTION 4: Example Analysis #1 - Simple Segmentation
# ============================================================================

print('\n' + '=' * 80)
print('EXAMPLE ANALYSIS #1: SIMPLE SEGMENTATION')
print('=' * 80 + '\n')

# Define segment
segment = df[
    (df.product_type == 'circle') &
    (df.base_temper == 'O') &
    (df.spec_thickness.between(0.80, 1.20))
].copy()

print(f'Segment: product="circle", temper="O", thickness 0.80-1.20 mm')
print(f'Result: {len(segment)} heats\n')

# Aggregate by alloy
summary = (
    segment
    .groupby(['alloy_code'], as_index=False)
    .agg(
        n=('heat_id', 'size'),
        avg_uts=('uts_value', 'mean'),
        sd_uts=('uts_value', 'std'),
        avg_ys=('ys_value', 'mean'),
    )
    .round({'avg_uts': 1, 'sd_uts': 1, 'avg_ys': 1})
    .sort_values('n', ascending=False)
)

print('Summary by alloy (reusable for reporting):')
display(summary)
print('\nInsight: UTS varies by alloy; consistency within each alloy is stable')

## Section 5: Example Analysis #2 - Quick Visualization

Two simple charts to validate data quality.

In [ ]:
# ============================================================================
# SECTION 5: Example Analysis #2 - Quick Visualization
# ============================================================================

print('\n' + '=' * 80)
print('EXAMPLE ANALYSIS #2: QUICK VISUALIZATION')
print('=' * 80 + '\n')

ALLOY = '3004'
# single_alloy = segment[segment['alloy_code'] == ALLOY]
SEGMENT_TITLE = 'O-temper circles, 0.80-1.20 mm'


fig, ax = plot_uts_histogram(segment,'ys_value',alloy = ALLOY,segment_name=SEGMENT_TITLE)
plt.show()

In [ ]:
# Boxplot: Compare across alloys
print('\nBoxplot: UTS distribution by alloy (same segment)\n')

fig, ax = plot_uts_boxplot( 
    segment,
    group_col='alloy_code',
    metric_col = 'uts_value',
    segment_name=SEGMENT_TITLE)

plt.show()

## Section 6: Sanity Checks

Quick validation that the dataset is complete and within expected ranges.

In [ ]:
# ============================================================================
# SECTION 6: Sanity Checks
# ============================================================================

print('\n' + '=' * 80)
print('SANITY CHECKS')
print('=' * 80 + '\n')

print('Data completeness:')
print(f'  Total heats: {len(df)}')
print(f'  Missing UTS: {df['uts_value'].isna().sum()}')
print(f'  Missing YS: {df['ys_value'].isna().sum()}')
print(f'  Missing YS: {df['elongation_value'].isna().sum()}')

print('\nAlloy distribution:')
print(df['alloy_code'].value_counts().sort_index())

print('\nTemper distribution:')
print(df['base_temper'].value_counts().sort_index())

print('\nUTS statistics:')
print(f'  Range: {df['uts_value'].min():.1f} - {df['uts_value'].max():.1f} MPa')
print(f'  Mean ± Std: {df['uts_value'].mean():.1f} ± {df['uts_value'].std():.1f} MPa')

print('\n✓ Dataset complete and within expected ranges')

## Section 7: Closing Notes

### What This Demonstrated

1. **Semantic layer as analytical interface:** Simple joins using `heat_id`
2. **Grain validation:** Heat-level contracts are explicit
3. **Analysis-ready dataset:** Built in 3 lines (pivot + 2 merges)
4. **Reproducible consumption:** CSV or PostgreSQL, same result

### Key Insight

**The semantic layer abstracts away data origin.** Analyst doesn't care whether data comes from CSV or live database. The interface is the same.

### Why This Matters for SC01

- Data quality is enforced upstream (not downstream)
- Analyses are comparable because semantics are stable
- Reproducibility is guaranteed because grain contracts are explicit
- Downstream analyses (SC02-SC05) can trust the foundation

### What Comes Next

- **SC02:** Does chemistry contain predictive signal for UTS?
- **SC03:** Do models generalize across alloy systems?
- **SC04:** Do process variables improve robustness?
- **SC05:** How do we translate models into decision tools?

All downstream studies use this same semantic layer, ensuring consistent, defensible analysis.